# [Random Pick with Weight](https://leetcode.com/problems/random-pick-with-weight/)
- **Given**:
    + Array `w[i]`, base 0
- **Return**: index `i` with a probability to pick = `w[i]` / `sum(w[i])`

#### Example

```
w = [0]1 [1]3

Chance to pick 0: P(0) = 1 / 4
Chance to pick 1: P(1) = 3 / 4
```

#### Constraints
- $1 \leq \text{w.length} \leq 10^4$

# Solution
## 1. Resevoir Sampling - O(N)
#### Algorithm (Suit for streaming data O(1) space)

```py
run_W = W
    # i=1, run_W = W
    # i=2, run_W = W - w[1]
    # i=3, run_W = W - w[1] - w[2]

for w[i] in w: 
    randomly pick and return i with P = w[i] / run_W
    run_W -= w[i]
```

#### Proof
- The chance for idx=1 to be picked

$$\begin{split}
    P\left(1\right) = \frac{w_1}{W}
\end{split}$$

- The chance for idx=2 to be picked (1 not picked, 2 picked)

$$\begin{split}
    P\left(2\right) &= \left(1 - \frac{w_1}{W}\right)\frac{w_2}{W-w_1} \\
        &= \frac{W-w_1}{W} \frac{w_2}{W-w_1} \\
        &= \frac{w_2}{W}
\end{split}$$

- The chance for idx=3 to be picked (1 not picked, 2 not picked, 3 picked)

$$\begin{split}
    P\left(3\right) &= \left(1 - \frac{w_1}{W}\right) \left(1 - \frac{w_2}{W-w_1}\right) \frac{w_3}{W-w_1-w_2} \\
        &= \frac{W-w_1}{W} \frac{W-w_1-w_2}{W-w_1} \frac{w_3}{W-w_1-w_2} \\
        &= \frac{w_3}{W}
\end{split}$$

#### Code

```Cpp
class Solution {
private:
    const vector<int> &_w;
    int _W;
public:
    Solution(const vector<int> &weights): _w(weights) {
        _W = accumulate(_w.begin(), _w.end(), 0);
    }

    int pickIndex() {
        int run_W = _W;
        for(int idx=0; idx<_w.size(); ++idx) {
            // Pick idx with P = _w[idx] / run_W
            int dice = rand() % run_W;
            if(dice < _w[idx]) return idx;

            // update run_W
            run_W -= _w[idx];
        }

        assert(false);
        return -1;
    }
};
```


## 2. Uniform segment
#### Idea
- Generate a uniform segment, length = W
- Use bin search to lookup for idx

```
|----|-------|----------|------------|
0    w0      w1         w2           w_(N-1)
```

#### Code

```Cpp
class Solution {
private:
    vector<int> _prefix_w;
    int _W;
public:
    Solution(const vector<int> &weights) {
        _prefix_w.assign(weights.size()+1, 0);
        _W = 0;

        for(int i=0; i<weights.size(); ++i) {
            _prefix_w[i+1] = _prefix_w[i] + weights[i];
            _W += weights[i];
        }
    }

    int pickIndex() {
        int dice = rand() % _W;

        // seg[idx] <= dice
        auto it = prev(upper_bound(_prefix_w.begin(), _prefix_w.end(), dice));
        return distance(_prefix_w.begin(), it);
    }
};
```